Second part of the project is to implement the same solution to the problem but with different approach. This time, I used a PSO algorithm from pyswarms to solve it.

In [142]:
import networkx as nx
import random
import numpy as np
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
import matplotlib.pyplot as plt
import time

Function that generates a graph with given number of nodes. There is a 30% chance that a node will have an edge to another node. Range of weights for each edge is [1, 6]. For testing purposes, the function also returns the longest path in the graph so we can later check if the algorithm found the ideal solution.

In [143]:
def generate_graph(number_of_nodes):
    G = nx.DiGraph()

    # Adding nodes
    G.add_nodes_from(range(number_of_nodes))

    # Adding edges with random weights between 1 and 6
    for i in range(number_of_nodes - 1):
        for j in range(i+1, number_of_nodes):
            if random.random() < 0.3:
                weight = random.randint(1, 6)
                G.add_edge(i, j, weight=weight)
                
    longest_path = nx.dag_longest_path(G, weight='weight')
    
    return (G, longest_path)

In [144]:
def draw_digraph(graph):
    pos = nx.kamada_kawai_layout(graph)
    nx.draw(graph, pos=pos, with_labels=True)
    labels = nx.get_edge_attributes(graph, 'weight')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)
    plt.show()

This function generates a matrix of connections between nodes. Row represents the first node and column represents the second node. The value of the cell is the weight of the connection between the nodes. If there is no connection between the nodes, the value of the cell is None.

In [145]:
def graph_to_distance_matrix(G):
    number_of_nodes = G.number_of_nodes()
    matrix = [[None for _ in range(number_of_nodes)] for _ in range(number_of_nodes)]

    for i in range(number_of_nodes - 1):
        for j in range(i + 1, number_of_nodes):
            if G.has_edge(i, j):
                matrix[i][j] = G[i][j]['weight']

    return matrix

In [146]:
# Example: [0, 1, 1, 0, 3, 3, 3, 2] -> [0, 1, 0, 3, 2]

def delete_subsequent_duplicates(list):
	return [list[i] for i in range(len(list)) if i == 0 or list[i] != list[i - 1]]

This function normalizes given path so it represents integers (nodes) instead of floats interpreted by PSO algorithm.

In [147]:
def normalize_position(position):
	return delete_subsequent_duplicates([int(p) for p in position])

This function clears the path so it doesn't contain any connections that are not valid using the given matrix of connections.

In [148]:
def clear_path(path, distance):
    current_node = path[0]
    cleared_path = [current_node]
    for node in path[1:]:
        if distance[current_node][node] is None:
            continue
        cleared_path.append(node)
        current_node = node
    return cleared_path
    

Fitness function iters through every positions provided by PSO. In each iteration it adds the weight of the connection between the current node and the next node to the total weight. In the end, it returns the total weight of the path divided by the length of the path. This way, the algorithm will try to find the shortest path (even if the wight is the highest possible). PSO searches for minimum by default, so we have to return an array of negative values.

In [149]:
def generate_fitness_function(distance_matrix):
    
    def fitness_function(positions):
        costs = []

        for position in positions:
            path = clear_path(normalize_position(position), distance_matrix)

            current_node = path[0]
            path_cost = 0
            for next_node in path[1:]:
                path_cost += distance_matrix[current_node][next_node]
                current_node = next_node

            costs.append(path_cost / (1 + 0.1 *len(path))) # Punish longer pahts (not singnificantly because it might break the algorithm)
        return np.array(costs) * -1
    
    return fitness_function

Declare the number of nodes in a graph:

In [150]:
number_of_nodes = 8

In [151]:
# Particle swarm optimization
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
n_particles = 700
n_iterations = 900


times = []
correct_solutions = 0
iterations = 5

# We run the algorithm multiple times, each time generating a new graph and its ideal solution.
for i in range(iterations):
    # Generate graph and ideal solution
    G, ideal_longest_path = generate_graph(number_of_nodes)
    
    # Define bounds
    constraints = (np.zeros(G.number_of_nodes()), (G.number_of_nodes() - 0.0000001) * np.ones(G.number_of_nodes()))

    print("Ideal longest path:", ideal_longest_path)

    # Convert graph to distance matrix
    distance_matrix = graph_to_distance_matrix(G)
    
    print("Run ", i+1)
    start = time.time()
    optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=G.number_of_nodes(), options=options, bounds=constraints)
    best_cost, best_pos = optimizer.optimize(generate_fitness_function(distance_matrix), iters=n_iterations)
    end = time.time()
    best_path = clear_path(normalize_position(best_pos), distance_matrix)

    # If the solution is correct, we add the time it took to the list of times and increase the correct solutions counter.
    if best_path == ideal_longest_path:
        print("Found ideal solution!")
        correct_solutions += 1
        times.append(end - start)
        

    print("Best solution (path):",  best_path)
    print("Fitness:", best_cost)
    print("-------------------------------------------")
    
average_time = 0
for time in times:
    average_time += time
if len(times) == 0:
    print("No ideal solutions were found, so the average time of finding one is 0.")
else:
    average_time = average_time / (len(times) )
    print("Average time of finding a single ideal solution: ", average_time, " s")
    
# Accuracy is the percentage of times the algorithm found the ideal solution. We check it with the "nx.dag_longest_path" buit-in function.
accuracy = (correct_solutions / iterations) * 100
print("Accuracy: ", accuracy, " %")

2023-04-11 03:46:22,765 - pyswarms.single.global_best - INFO - Optimize for 900 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


Ideal longest path: [0, 2, 6]
Run  1


pyswarms.single.global_best: 100%|██████████|900/900, best_cost=-7.69
2023-04-11 03:46:27,580 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -7.692307692307692, best pos: [3.02048761 4.2319803  5.59870465 4.52904429 5.14468829 1.03490308
 0.87616459 7.60595342]
2023-04-11 03:46:27,629 - pyswarms.single.global_best - INFO - Optimize for 900 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


Best solution (path): [3, 5, 7]
Fitness: -7.692307692307692
-------------------------------------------
Ideal longest path: [0, 3, 6]
Run  2


pyswarms.single.global_best: 100%|██████████|900/900, best_cost=-7.69
2023-04-11 03:46:34,177 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -7.692307692307692, best pos: [0.07775631 4.16130543 3.2399454  2.34374941 5.53855542 3.51486117
 6.15870012 5.09118577]
2023-04-11 03:46:34,260 - pyswarms.single.global_best - INFO - Optimize for 900 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


...

Found ideal solution!
Best solution (path): [3, 7]
Fitness: -4.166666666666667
-------------------------------------------
Ideal longest path: [0, 1, 3, 5]
Run  4


pyswarms.single.global_best: 100%|██████████|900/900, best_cost=-12.1
2023-04-11 03:46:45,751 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -12.142857142857144, best pos: [0.21923488 5.26768187 1.07761561 3.50579184 4.09762861 4.45078834
 5.92807501 1.1725282 ]
2023-04-11 03:46:45,772 - pyswarms.single.global_best - INFO - Optimize for 900 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}


Found ideal solution!
Best solution (path): [0, 1, 3, 5]
Fitness: -12.142857142857144
-------------------------------------------
Ideal longest path: [2, 4, 6, 7]
Run  5


pyswarms.single.global_best: 100%|██████████|900/900, best_cost=-10
2023-04-11 03:46:50,682 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -10.0, best pos: [2.5438818  4.28622307 6.2274907  7.39450861 2.89898325 2.94856435
 3.32798859 0.90913346]


Found ideal solution!
Best solution (path): [2, 4, 6, 7]
Fitness: -10.0
-------------------------------------------
Average time of finding a single ideal solution:  5.770875334739685  s
Accuracy:  80.0  %


Average time of finding ideal solutions and their percentage in all results for PSO (100 iterations):

In [152]:
#   ---------------------------------------------------------------------------------------------------------
#   |   Graph size   |   n_particles   |   n_iterations  |   Average time   |   Accuracy   |   Total time   | 
#   |-------------------------------------------------------------------------------------------------------|
#   |   8            |   800           |   900           |   4.81s          |   92%        |   8m 1s        |
#   |-------------------------------------------------------------------------------------------------------|
#   |   14           |   1000          |   1100          |   11.68s         |   71%        |   19m 31s      |
#   |-------------------------------------------------------------------------------------------------------|
#   |   20           |   1200          |   1500          |   20.34s         |   38%        |   33m 49s      |
#   ---------------------------------------------------------------------------------------------------------

Bibliography

[1] https://pyswarms.readthedocs.io/en/latest/index.html

[2] https://en.wikipedia.org/wiki/Particle_swarm_optimization